## Installing Libraries Needed

In [1]:
!python -m pip install pandas

In [2]:
!python -m pip install openpyxl


## Dataset Inventory Loader

In [3]:
import os
import re

base_path = r"G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey"

# Month ordering
month_order = {
    "January": 1, "February": 2, "March": 3, "April": 4,
    "May": 5, "June": 6, "July": 7, "August": 8,
    "September": 9, "October": 10, "November": 11, "December": 12
}

# Patterns
month_pattern = re.compile(
    r"(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER)",
    re.IGNORECASE
)
year_pattern = re.compile(r"(20\d{2})")

# Detect year folders from drive
year_folders = [
    f for f in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, f)) and f.isdigit()
]

print("Detected year folders:", sorted(year_folders))

inventory = {}

for year in sorted(year_folders):
    year_path = os.path.join(base_path, year)

    # Accept both CSV and XLSX
    data_files = [
        f for f in os.listdir(year_path)
        if f.lower().endswith(".csv") or f.lower().endswith(".xlsx")
    ]

    inventory[year] = {}

    for file in data_files:
        upper = file.upper()

        # Detect type
        if upper.endswith(".XLSX"):
            filetype = "metadata"  # XLSX = metadata
        else:
            filetype = "survey"    # CSV = survey

        # Detect month
        month_match = month_pattern.search(upper)
        month = (
            month_match.group(1).capitalize()
            if month_match
            else "Unmatched"
        )

        # Detect year inside filename
        year_match = year_pattern.search(upper)
        file_year = year_match.group(1) if year_match else "UNKNOWN"

        # Store into inventory
        if month not in inventory[year]:
            inventory[year][month] = []

        inventory[year][month].append({
            "filename": file,
            "filetype": filetype,
            "file_year": file_year
        })

# Print clean summary
print("\n=== DATASET INVENTORY SUMMARY ===\n")

for yr in sorted(inventory.keys()):
    print(f"Year {yr}:")

    sorted_months = sorted(
        inventory[yr].keys(),
        key=lambda m: month_order.get(m, 99)
    )

    for month in sorted_months:
        print(f"  {month}:")
        for item in inventory[yr][month]:
            print(f"    {item['filename']} ({item['filetype']})")

    print()


Detected year folders: ['2018', '2019', '2022', '2023', '2024']

=== DATASET INVENTORY SUMMARY ===

Year 2018:
  January:
    JANUARY_2018_METADATA.xlsx (metadata)
    JANUARY_2018.CSV (survey)
    ~$JANUARY_2018_METADATA.xlsx (metadata)
  April:
    APRIL_2018_METADATA.xlsx (metadata)
    APRIL_2018.CSV (survey)
    ~$APRIL_2018_METADATA.xlsx (metadata)
  July:
    JULY_2018.CSV (survey)
    JULY_2018_METADATA.xlsx (metadata)
  October:
    OCTOBER_2018_METADATA.xlsx (metadata)
    OCTOBER_2018.CSV (survey)

Year 2019:
  January:
    JANUARY_2019_METADATA.xlsx (metadata)
    JANUARY_2019.CSV (survey)
  April:
    APRIL_2019.CSV (survey)
    APRIL_2019_METADATA.xlsx (metadata)
  July:
    JULY_2019.CSV (survey)
    JULY_2019_METADATA.xlsx (metadata)
  October:
    OCTOBER_2019.CSV (survey)
    OCTOBER_2019_METADATA.xlsx (metadata)

Year 2022:
  January:
    JANUARY_2022_METADATA.xlsx (metadata)
    JANUARY_2022.csv (survey)
  February:
    FEBRUARY_2022_METADATA.xlsx (metadata)
    FEB

## Load Dataset Function

In [4]:
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load a dataset from the inventory.

    year: str, e.g., "2018"
    month: str, e.g., "January"
    filetype: "survey" or "metadata"
    sheet_number: 0(sheet 1) or 1(sheet 2)
    """
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )
    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])
    
    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)
    
    if sheet_number is not None:
        return pd.read_excel(file_path, sheet_name=sheet_number)
    
    return pd.read_excel(file_path)

Sample: January 2018 Survey

In [5]:
import pandas as pd

# Load the survey sheet of January 2018 metadata
jan_2018_survey = load_dataset("2018", "January","survey")

# View the first few rows
jan_2018_survey.head()

,PUFREG,PUFPRV,PUFPRRCD,PUFHHNUM,PUFURB2K10,PUFPWGTPRV,PUFSVYMO,PUFSVYYR,PUFPSU,PUFRPL,...,PUFC33_WEEKS,PUFC34_WYNOT,PUFC35_LTLOOKW,PUFC36_AVAIL,PUFC37_WILLING,PUFC38_PREVJOB,PUFC40_POCC,PUFC41_WQTR,PUFC43_QKB,PUFNEWEMPSTAT
0,14,1,100,1,2,124.9425,1,2018,140,32,...,,6,,,,1,52,2,,3
1,14,1,100,1,2,131.2126,1,2018,140,32,...,,,,,,,,1,01,1
2,14,1,100,1,2,142.0464,1,2018,140,32,...,,,,,,,,1,01,1
3,14,1,100,1,2,138.2958,1,2018,140,32,...,,,,,,,,,,
4,14,1,100,2,2,195.4152,1,2018,140,32,...,,,,,,,,1,41,1


## Metadata Sheet 1

<H5> Sample: January 2018 Metadata Sheet 1 (Raw) </H5>

In [6]:
# Load the first sheet of January 2018 metadata
january_2018_metadata_sheet1 = load_dataset("2018", "January", "metadata", 0)

# View the first few rows
print("=== January 2018 Metadata Sheet 1 (Raw) ===")
january_2018_metadata_sheet1.head()


=== January 2018 Metadata Sheet 1 (Raw) ===


,QUEST,Questionnaire,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,_IDS0,(Id Items),NaN,NaN
1,NaN,NaN,NaN,NaN,PUFREG,Region
2,NaN,NaN,NaN,NaN,PUFPRV,Province
3,NaN,NaN,NaN,NaN,PUFPRRCD,Province Recode
4,NaN,NaN,NaN,NaN,PUFHHNUM,Household Unique Sequential Number


#### Reshaping Metadata Sheet 1

In [7]:
import pandas as pd

def extract_variables(df):
    """
    Extract variable names and descriptions from metadata Sheet 1 (variable dictionary).
    Automatically reads the 4th and 5th columns (E and F in Excel) where variables and descriptions reside.
    
    Returns a clean DataFrame with columns ['Variable', 'Description'].
    """
    
    # Select the 4th and 5th columns (index 4 and 5)
    df_vars = df.iloc[:, 4:6].copy()
    
    # Rename columns
    df_vars.columns = ['Variable', 'Description']
    
    # Drop rows where 'Variable' is empty or NaN
    df_vars = df_vars[df_vars['Variable'].notna() & (df_vars['Variable'].astype(str).str.strip() != '')]
    
    # Strip whitespace from values
    df_vars['Variable'] = df_vars['Variable'].astype(str).str.strip()
    df_vars['Description'] = df_vars['Description'].astype(str).str.strip()
    
    # Reset index
    df_vars = df_vars.reset_index(drop=True)
    
    return df_vars


### Metadata Sheet 1 Reshaped Saving Function

In [8]:
import os
import pandas as pd

def batch_process_sheet1_metadata(inventory, base_output_path):
    """
    Loops through the entire inventory, loads Sheet 1 of the metadata,
    reshapes it, and saves it into a structured folder hierarchy.
    
    Provides a text-based summary report for assurance.
    """
    
    # Counters for the summary report
    success_count = 0
    failure_count = 0
    skipped_count = 0
    errors_log = []

    # 1. Define and Create the Main Parent Folder
    main_folder_name = "Metadata Sheet 1 CSV's"
    main_folder_path = os.path.join(base_output_path, main_folder_name)
    os.makedirs(main_folder_path, exist_ok=True)
    
    print("--- STARTING BATCH PROCESS ---")
    print(f"Target Directory: {main_folder_path}")
    print("-" * 50)

    # 2. Iterate through Years in the Inventory
    for year, months_data in inventory.items():
        
        # Create the Year Subfolder
        year_folder_path = os.path.join(main_folder_path, year)
        os.makedirs(year_folder_path, exist_ok=True)
        
        # 3. Iterate through Months in that Year
        for month, files_list in months_data.items():
            
            if month == "Unmatched":
                continue
            
            # Check for metadata file existence
            has_metadata = any(f.get('filetype') == 'metadata' for f in files_list)
            
            if has_metadata:
                try:
                    # A. Load the Data (Sheet 0 = Sheet 1)
                    raw_df = load_dataset(year, month, "metadata", 0)
                    
                    # B. Reshape the Data
                    clean_df = extract_variables(raw_df)
                    
                    # C. Save to CSV
                    filename = f"Sheet1_{month}_{year}.csv"
                    full_save_path = os.path.join(year_folder_path, filename)
                    
                    clean_df.to_csv(full_save_path, index=False)
                    
                    # Print confirmation for this specific file
                    print(f"[OK] Saved: {year}/{filename}")
                    success_count += 1
                    
                except Exception as e:
                    print(f"[ERROR] Failed {month} {year}: {e}")
                    errors_log.append(f"{month} {year}: {str(e)}")
                    failure_count += 1
            else:
                skipped_count += 1

    # 4. Final Assurance Report
    print("\n" + "="*40)
    print("      PROCESSING SUMMARY REPORT")
    print("="*40)
    print(f"Total Successfully Saved: {success_count}")
    print(f"Total Failed:             {failure_count}")
    print(f"Total Skipped (No File):  {skipped_count}")
    print("-" * 40)
    
    if failure_count == 0:
        print("STATUS: COMPLETE SUCCESS")
        print(f"All files are now located in: {main_folder_path}")
        print("Google Drive is syncing these files now.")
    else:
        print("STATUS: COMPLETED WITH ERRORS")
        print("Check the errors log above.")
        if errors_log:
            print("\nError Details:")
            for err in errors_log:
                print(f" - {err}")
    print("="*40)

In [9]:
# Run the processor
batch_process_sheet1_metadata(inventory, base_path)

--- STARTING BATCH PROCESS ---
Target Directory: G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\Metadata Sheet 1 CSV's
--------------------------------------------------
[OK] Saved: 2018/Sheet1_January_2018.csv
[OK] Saved: 2018/Sheet1_April_2018.csv
[OK] Saved: 2018/Sheet1_July_2018.csv
[OK] Saved: 2018/Sheet1_October_2018.csv
[OK] Saved: 2019/Sheet1_January_2019.csv
[OK] Saved: 2019/Sheet1_April_2019.csv
[OK] Saved: 2019/Sheet1_July_2019.csv
[OK] Saved: 2019/Sheet1_October_2019.csv
[OK] Saved: 2022/Sheet1_August_2022.csv
[OK] Saved: 2022/Sheet1_September_2022.csv
[OK] Saved: 2022/Sheet1_January_2022.csv
[OK] Saved: 2022/Sheet1_June_2022.csv
[OK] Saved: 2022/Sheet1_October_2022.csv
[OK] Saved: 2022/Sheet1_December_2022.csv
[OK] Saved: 2022/Sheet1_March_2022.csv
[OK] Saved: 2022/Sheet1_April_2022.csv
[OK] Saved: 2022/Sheet1_May_2022.csv
[OK] Saved: 2022/Sheet1_July_2022.csv
[OK] Saved: 2022/Sheet1_November_2022.csv
[OK] Saved: 2022/Sheet1_February_2022.c

#### Verifying if the variable and description counts of Reshaped Metadata Sheet 1 and Original matches

In [10]:
def batch_verify_sheet1_variable_and_description_count_verbose(inventory, base_path):
    """
    Iterates through all years and months in the inventory and compares
    total variables and descriptions in raw vs reshaped Sheet 1 metadata.
    Prints mismatches immediately, and returns a DataFrame with all results.
    """

    results = []

    for year, months_data in inventory.items():
        for month, files_list in months_data.items():
            if month == "Unmatched":
                continue  # Skip unmatched files

            # --- Load raw Sheet 1 ---
            try:
                raw_df = load_dataset(year, month, "metadata", sheet_number=0)
            except Exception as e:
                print(f"[ERROR] {month} {year}: Could not load raw Sheet 1 ({e})")
                results.append({
                    'Year': year,
                    'Month': month,
                    'Raw Variable Count': 'ERROR',
                    'Reshaped Variable Count': 'ERROR',
                    'Raw Description Count': 'ERROR',
                    'Reshaped Description Count': 'ERROR',
                    'Status': f'FAIL (Raw load error: {e})'
                })
                continue

            # --- Load reshaped CSV Sheet 1 ---
            reshaped_file_path = os.path.join(
                base_path, "Metadata Sheet 1 CSV's", year, f"Sheet1_{month}_{year}.csv"
            )
            if not os.path.exists(reshaped_file_path):
                print(f"[ERROR] {month} {year}: Reshaped Sheet 1 CSV missing!")
                results.append({
                    'Year': year,
                    'Month': month,
                    'Raw Variable Count': 'ERROR',
                    'Reshaped Variable Count': 'ERROR',
                    'Raw Description Count': 'ERROR',
                    'Reshaped Description Count': 'ERROR',
                    'Status': 'FAIL (Reshaped CSV missing)'
                })
                continue

            reshaped_df = pd.read_csv(reshaped_file_path)

            # --- Count non-empty variables and descriptions ---
            raw_vars = raw_df.iloc[:, 4].dropna().astype(str).str.strip()
            raw_vars = raw_vars[raw_vars != '']
            raw_descs = raw_df.iloc[:, 5].dropna().astype(str).str.strip()
            raw_descs = raw_descs[raw_descs != '']

            reshaped_vars = reshaped_df['Variable'].astype(str).str.strip()
            reshaped_vars = reshaped_vars[reshaped_vars != '']
            reshaped_descs = reshaped_df['Description'].astype(str).str.strip()
            reshaped_descs = reshaped_descs[reshaped_descs != '']

            # --- Check if both counts match ---
            status = "PASS" if (len(raw_vars) == len(reshaped_vars) and len(raw_descs) == len(reshaped_descs)) else "FAIL"

            if status == "FAIL":
                # Immediate print for any mismatch
                print(f"[MISMATCH] {month} {year} - Variables: {len(raw_vars)} vs {len(reshaped_vars)}, "
                      f"Descriptions: {len(raw_descs)} vs {len(reshaped_descs)}")

            results.append({
                'Year': year,
                'Month': month,
                'Raw Variable Count': len(raw_vars),
                'Reshaped Variable Count': len(reshaped_vars),
                'Raw Description Count': len(raw_descs),
                'Reshaped Description Count': len(reshaped_descs),
                'Status': status
            })

    return pd.DataFrame(results).sort_values(['Year', 'Month']).reset_index(drop=True)

In [11]:
# Run the Sheet 1 verifier
verification_df = batch_verify_sheet1_variable_and_description_count_verbose(inventory, base_path)

# Print a header and show the first few rows
print("=== Sheet 1 Metadata Variables and Descriptions (Raw vs Reshaped) ===")
verification_df.head()

=== Sheet 1 Metadata Variables and Descriptions (Raw vs Reshaped) ===


,Year,Month,Raw Variable Count,Reshaped Variable Count,Raw Description Count,Reshaped Description Count,Status
0,2018,April,50,50,50,50,PASS
1,2018,January,50,50,50,50,PASS
2,2018,July,51,51,51,51,PASS
3,2018,October,51,51,51,51,PASS
4,2019,April,49,49,49,49,PASS


Checking January 2018 Metadata Reshaped Sheet 1

In [12]:
# Load metadata Sheet 1
January_metadata = load_dataset("2018", "January", "metadata", 0)

# Call your function
variables_df = extract_variables(January_metadata)

# View results
variables_df.head()

,Variable,Description
0,PUFREG,Region
1,PUFPRV,Province
2,PUFPRRCD,Province Recode
3,PUFHHNUM,Household Unique Sequential Number
4,PUFURB2K10,2010Urban-RuralFIES


Checking August 2024 Metadata Reshaped Sheet 1

In [13]:
# Load metadata Sheet 1
August_2024_metadata = load_dataset("2024", "August", "metadata", 0)

# Call your function
variables_df = extract_variables(August_2024_metadata)

# View results
variables_df.head()

,Variable,Description
0,PUFHHNUM,Household Unique Sequential Number
1,PUFPWGTPRV,Final Weight Based on Projection
2,PUFSVYMO,Survey Month
3,PUFSVYYR,Survey Year
4,PUFPSU,Psu Number


## Metadata Sheet 2 Function

<H5> Sample: January 2018 Metadata Sheet 2 (Raw)</H5>

In [14]:
# Load the second sheet of January 2018 metadata
january_2018_metadata_sheet2 = load_dataset("2018", "January", "metadata", 1)

# View the first few rows
print("=== January 2018 Metadata Sheet 2 (Raw) ===")
january_2018_metadata_sheet2.head()

=== January 2018 Metadata Sheet 2 (Raw) ===


,PUFREG_VS1,Region,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,National Capital Region,13,NaN,NaN
1,NaN,NaN,Cordillera Administrative Region,14,NaN,NaN
2,NaN,NaN,Region I - Ilocos Region,1,NaN,NaN
3,NaN,NaN,Region II - Cagayan Valley,2,NaN,NaN
4,NaN,NaN,Region III - Central Luzon,3,NaN,NaN


### Reshaping Metadata Sheet 2

In [15]:
import os
import pandas as pd


def reshape_sheet2_robust(df):
    """
    Convert metadata Sheet 2 (the values dictionary) into a clean, long-format table.

    This function reads the sheet exactly as it appears in Excel, without:
    - Assuming any header row
    - Auto-filling missing values
    - Inferencing min/max values
    - Guessing variable names

    Sheet 2 typically has this layout:
        Column A = Variable name (only appears once per block)
        Column B = Variable description (blank except at the start of a block)
        Column C = Label for each value (required)
        Column D = Minimum value (optional)
        Column E = Maximum value (optional)
        Column F+ = Additional text or category notes (optional)

    The function processes rows in order and:
        - Carries forward the most recent non-empty variable name (Column A)
        - Carries forward the most recent non-empty description (Column B)
        - Creates one output row per value label (Column C)
        - Leaves missing min/max/additional values as 0
        - Reads extra info (Column F onward) if present

    Returns:
        A clean pandas DataFrame with columns:
            Variable
            Description
            Label
            min_value
            max_value
            additional_value
    """

    reshaped = []

    # Ensure all blanks are handled consistently
    df = df.fillna('').astype(str)

    # Initialize with the first variable and description
    current_var = df.iloc[0, 0].strip() or 'UNKNOWN_VAR'
    current_desc = df.iloc[0, 1].strip() or ''

    # Iterate row-by-row
    for idx, row in df.iterrows():
        # ---- Column A: Variable name ----
        var_candidate = row.iloc[0].strip()
        if var_candidate:
            current_var = var_candidate

        # ---- Column B: Description ----
        desc_candidate = row.iloc[1].strip()
        if desc_candidate:
            current_desc = desc_candidate

        # ---- PRE-READ Columns D, E, F (Values) ----
        raw_min = row.iloc[3].strip()
        raw_max = row.iloc[4].strip()
        
        # Look for extra values (Column F+)
        extra = '0'
        if len(row) > 5:
            for j in range(5, len(row)):
                extra_candidate = row.iloc[j].strip()
                if extra_candidate:
                    extra = extra_candidate
                    break

        # ---- Column C: Label ----
        label = row.iloc[2].strip()

        # FIX: Don't just continue. Check if values exist.
        if not label:
            # If label is missing BUT we have min, max, or extra -> It's a valid row
            if raw_min or raw_max or extra != '0':
                label = '0'  # Assign default label
            else:
                continue     # Skip only if truly empty

        # ---- Finalize Min/Max ----
        min_value = raw_min if raw_min else '0'
        max_value = raw_max if raw_max else '0'

        # ---- Append clean record ----
        reshaped.append({
            "Variable": current_var,
            "Description": current_desc,
            "Label": label,
            "min_value": min_value,
            "max_value": max_value,
            "additional_value": extra
        })

    return pd.DataFrame(reshaped)


# ============================================================
#   load_dataset()
# ============================================================
def load_dataset(year, month, filetype="survey", sheet_number=None):
    """
    Load any dataset (survey or metadata) from the file inventory.

    • For SURVEY CSV: normal pandas.read_csv()
    • For METADATA Excel: read with no header, reshape Sheet 2 automatically
    """
    # Retrieve file information from inventory
    file_info = next(
        (f for f in inventory[year][month] if f["filetype"] == filetype),
        None
    )

    if not file_info:
        raise ValueError(f"No {filetype} file found for {month} {year}")

    file_path = os.path.join(base_path, year, file_info["filename"])

    if filetype == "survey":
        return pd.read_csv(file_path, low_memory=False)

    # Metadata Excel — always read with no header
    df = pd.read_excel(file_path, sheet_name=sheet_number, header=None)

    # Automatic reshaping ONLY for metadata Sheet 2
    if sheet_number == 1:
        df = reshape_sheet2_robust(df)

    return df


### Metadata Sheet 2 Reshaped Saving Function

In [16]:
import os
import pandas as pd

def batch_process_sheet2_metadata(inventory, base_output_path):
    """
    Loops through the inventory to process 'Sheet 2' (Value Codes).
    """
    
    # Counters for the summary report
    success_count = 0
    failure_count = 0
    skipped_count = 0
    errors_log = []

    # 1. Define Main Folder Name
    main_folder_name = "Metadata Sheet 2 CSV's"
    main_folder_path = os.path.join(base_output_path, main_folder_name)
    os.makedirs(main_folder_path, exist_ok=True)
    
    print("--- STARTING BATCH PROCESS (SHEET 2) ---")
    print(f"Target Directory: {main_folder_path}")
    print("-" * 50)

    # 2. Iterate through Inventory
    for year, months_data in inventory.items():
        
        # Create Year Subfolder
        year_folder_path = os.path.join(main_folder_path, year)
        os.makedirs(year_folder_path, exist_ok=True)
        
        for month, files_list in months_data.items():
            # Skip unmatched files
            if month == "Unmatched":
                continue
            
            # Check if metadata exists for this month
            has_metadata = any(f.get('filetype') == 'metadata' for f in files_list)
            
            if has_metadata:
                try:
                    # A. Load & Reshape
                    # Your load_dataset function handles the cleaning internally
                    clean_df = load_dataset(year, month, "metadata", 1)
                    
                    # B. Generate Filename
                    filename = f"Sheet2_{month}_{year}.csv"
                    full_save_path = os.path.join(year_folder_path, filename)
                    
                    # C. Save
                    clean_df.to_csv(full_save_path, index=False)
                    
                    print(f"[OK] Saved: {year}/{filename}")
                    success_count += 1
                    
                except Exception as e:
                    print(f"[ERROR] Failed {month} {year}: {e}")
                    errors_log.append(f"{month} {year}: {str(e)}")
                    failure_count += 1
            else:
                skipped_count += 1

    # 3. Final Report
    print("\n" + "="*40)
    print("      SHEET 2 PROCESSING SUMMARY")
    print("="*40)
    print(f"Total Saved:    {success_count}")
    print(f"Total Failed:   {failure_count}")
    print(f"Total Skipped:  {skipped_count}")
    print("-" * 40)
    
    if failure_count == 0:
        print("STATUS: COMPLETE SUCCESS")
        print(f"Files are syncing to: {main_folder_path}")
    else:
        print("STATUS: COMPLETED WITH ERRORS")
        for err in errors_log:
            print(f" - {err}")
    print("="*40)

In [17]:
# Run the processor
# (Requires 'inventory' and 'load_dataset' to be defined in your environment)
batch_process_sheet2_metadata(inventory, base_path)

--- STARTING BATCH PROCESS (SHEET 2) ---
Target Directory: G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\Metadata Sheet 2 CSV's
--------------------------------------------------
[OK] Saved: 2018/Sheet2_January_2018.csv
[OK] Saved: 2018/Sheet2_April_2018.csv
[OK] Saved: 2018/Sheet2_July_2018.csv
[OK] Saved: 2018/Sheet2_October_2018.csv
[OK] Saved: 2019/Sheet2_January_2019.csv
[OK] Saved: 2019/Sheet2_April_2019.csv
[OK] Saved: 2019/Sheet2_July_2019.csv
[OK] Saved: 2019/Sheet2_October_2019.csv
[OK] Saved: 2022/Sheet2_August_2022.csv
[OK] Saved: 2022/Sheet2_September_2022.csv
[OK] Saved: 2022/Sheet2_January_2022.csv
[OK] Saved: 2022/Sheet2_June_2022.csv
[OK] Saved: 2022/Sheet2_October_2022.csv
[OK] Saved: 2022/Sheet2_December_2022.csv
[OK] Saved: 2022/Sheet2_March_2022.csv
[OK] Saved: 2022/Sheet2_April_2022.csv
[OK] Saved: 2022/Sheet2_May_2022.csv
[OK] Saved: 2022/Sheet2_July_2022.csv
[OK] Saved: 2022/Sheet2_November_2022.csv
[OK] Saved: 2022/Sheet2_Febru

#### Verifying if the variable counts of Reshaped Metadata Sheet 2 and Original matches

In [18]:
import os
import pandas as pd

def batch_verify_sheet2_variable_and_label_count(inventory, base_path):
    """
    Batch verify Sheet 2 metadata (values dictionary) across years/months.
    Compares:
      • Unique variable count (raw vs reshaped)
      • Label count per variable (raw vs reshaped)
    Prints mismatches immediately and returns a summary DataFrame.
    """
    all_results = []

    for year, months_data in inventory.items():
        for month, files_list in months_data.items():
            if month == "Unmatched":
                continue

            # --- Load raw Sheet 2 ---
            try:
                raw_df = load_dataset(year, month, "metadata", sheet_number=1)
            except Exception as e:
                print(f"[ERROR] {month} {year}: Could not load raw Sheet 2 ({e})")
                continue

            # --- Load reshaped Sheet 2 CSV ---
            reshaped_path = os.path.join(base_path, "Metadata Sheet 2 CSV's", year, f"Sheet2_{month}_{year}.csv")
            if not os.path.exists(reshaped_path):
                print(f"[ERROR] {month} {year}: Reshaped Sheet 2 CSV missing!")
                continue

            reshaped_df = pd.read_csv(reshaped_path, dtype=str).fillna("")

            # --- Count unique variables ---
            raw_vars = raw_df.iloc[:, 0].astype(str).str.strip()
            raw_vars = raw_vars[raw_vars != '']  # ignore empty
            raw_unique_vars = pd.Index(raw_vars).unique()

            resh_vars = reshaped_df['Variable'].astype(str).str.strip()
            resh_unique_vars = pd.Index(resh_vars).unique()

            # Check variable count mismatch
            variable_mismatch = len(raw_unique_vars) != len(resh_unique_vars)
            if variable_mismatch:
                print(f"[VARIABLE COUNT MISMATCH] {month} {year}: Raw={len(raw_unique_vars)}, Reshaped={len(resh_unique_vars)}")

            # --- Count labels per variable ---
            label_mismatches = []

            for var in raw_unique_vars:
                # Raw: select rows matching variable
                raw_rows = raw_df[raw_df.iloc[:, 0].astype(str).str.strip() == var]

                # Count non-empty label cells safely
                raw_label_count = raw_rows.iloc[:, 2:6].astype(str).apply(lambda x: x.str.strip().ne('').any(), axis=1).sum()

                # Reshaped: count rows per variable
                resh_label_count = reshaped_df[reshaped_df['Variable'].astype(str).str.strip() == var].shape[0]

                if raw_label_count != resh_label_count:
                    label_mismatches.append({
                        "Variable": var,
                        "Raw_Label_Count": raw_label_count,
                        "Reshaped_Label_Count": resh_label_count
                    })

            # --- Print immediate label mismatches ---
            for m in label_mismatches:
                print(f"[LABEL COUNT MISMATCH] {month} {year} - Variable: {m['Variable']} | Raw={m['Raw_Label_Count']} vs Reshaped={m['Reshaped_Label_Count']}")

            # --- Record summary ---
            all_results.append({
                "Year": year,
                "Month": month,
                "Raw_Variable_Count": len(raw_unique_vars),
                "Reshaped_Variable_Count": len(resh_unique_vars),
                "Variable_Count_Status": "PASS" if not variable_mismatch else "FAIL",
                "Label_Count_Mismatches": len(label_mismatches)
            })

    # Return as DataFrame
    return pd.DataFrame(all_results).sort_values(['Year', 'Month']).reset_index(drop=True)


In [19]:
summary_df = batch_verify_sheet2_variable_and_label_count(inventory, base_path)
print("=== Sheet 2 Variable & Label Count Verification ===")
summary_df

=== Sheet 2 Variable & Label Count Verification ===


,Year,Month,Raw_Variable_Count,Reshaped_Variable_Count,Variable_Count_Status,Label_Count_Mismatches
0,2018,April,43,43,PASS,0
1,2018,January,46,46,PASS,0
2,2018,July,44,44,PASS,0
3,2018,October,52,52,PASS,0
4,2019,April,45,45,PASS,0
5,2019,January,44,44,PASS,0
6,2019,July,56,56,PASS,0
7,2019,October,56,56,PASS,0
8,2022,April,57,57,PASS,0
9,2022,August,46,46,PASS,0


#### A verifier to check if all variables, descriptions, labels, and value fields in the original Sheet 2 exist and match in the reshaped Sheet 2, regardless of row order

In [20]:
def verify_sheet2(original_df, reshaped_df):
    """
    Compare original Sheet 2 with reshaped version.
    Checks:
      • Same variables
      • Same descriptions
      • Same labels
      • Same min/max/additional values
      • Ignores row order
    """

    # Normalize to string
    original = original_df.fillna("").astype(str)
    reshaped = reshaped_df.fillna("").astype(str)

    # --- Extract original as dict ---
    def build_original_dict(df):
        data = {}
        current_var = ""
        current_desc = ""
        for _, row in df.iterrows():
            colA = row.iloc[0].strip()
            colB = row.iloc[1].strip()
            colC = row.iloc[2].strip()
            if colA:
                current_var = colA
            if colB:
                current_desc = colB
            if not colC:
                continue
            minv = row.iloc[3].strip() if len(row) > 3 else ""
            maxv = row.iloc[4].strip() if len(row) > 4 else ""
            extra = ""
            if len(row) > 5:
                for j in range(5, len(row)):
                    if row.iloc[j].strip():
                        extra = row.iloc[j].strip()
                        break
            if current_var not in data:
                data[current_var] = []
            data[current_var].append({
                "Description": current_desc,
                "Label": colC,
                "min_value": minv,
                "max_value": maxv,
                "additional_value": extra
            })
        return data

    orig_dict = build_original_dict(original)

    # --- Extract reshaped as dict (warning-free) ---
    resh_dict = {
        var: group.drop(columns="Variable").to_dict("records")
        for var, group in reshaped.groupby("Variable")
    }

    # --- Verification ---
    errors = []

    orig_vars = set(orig_dict.keys())
    resh_vars = set(resh_dict.keys())

    missing_vars = orig_vars - resh_vars
    extra_vars = resh_vars - orig_vars
    if missing_vars:
        errors.append(f"Missing variables in reshaped: {missing_vars}")
    if extra_vars:
        errors.append(f"Extra variables in reshaped: {extra_vars}")

    # Detailed label/content comparison
    for var in orig_vars & resh_vars:
        orig_records = orig_dict[var]
        resh_records = resh_dict[var]
        orig_set = {
            (d["Label"], d["min_value"], d["max_value"], d["additional_value"])
            for d in orig_records
        }
        resh_set = {
            (d["Label"], d["min_value"], d["max_value"], d["additional_value"])
            for d in resh_records
        }
        missing_rec = orig_set - resh_set
        extra_rec = resh_set - orig_set
        if missing_rec:
            errors.append(f"[{var}] Missing records: {missing_rec}")
        if extra_rec:
            errors.append(f"[{var}] Extra records: {extra_rec}")

    if not errors:
        return "SUCCESS: Reshaped metadata matches the original Sheet 2."
    else:
        return "MISMATCH FOUND:\n" + "\n".join(errors)


Checking for January 2018 Original vs Reshaped Metadata Sheet 2

In [21]:
# ===========================
# Caller: content-based verification with month/year in messages
# ===========================

# Load original metadata (Sheet 2)
original_df = load_dataset("2018", "January", "metadata", 1)

# Load reshaped metadata (automatically reshaped)
reshaped_df = load_dataset("2018", "January", "metadata", 1)

# Store month/year for messages
month = "January"
year = "2018"

# Call the verifier
result_message = verify_sheet2(original_df, reshaped_df)

# Customize message to include month/year
if "SUCCESS" in result_message:
    print(f"SUCCESS: Reshaped Sheet 2 metadata for {month} {year} matches the original.\n")
    print("Sample content verification (first 5 variables):\n")
    
    # Pick first 5 unique variables from reshaped
    sample_vars = reshaped_df["Variable"].unique()[:5]
    for var in sample_vars:
        sample_rows = reshaped_df[reshaped_df["Variable"] == var]
        print(f"Variable: {var}")
        print(sample_rows[["Label", "min_value", "max_value", "additional_value"]])
        print("-" * 50)
else:
    print(f"MISMATCH FOUND in {month} {year} metadata:\n")
    # Show first few rows from both original and reshaped for inspection
    orig_preview = original_df.fillna("").astype(str).head(10)
    resh_preview = reshaped_df.fillna("").astype(str).head(10)
    print("Original preview:")
    print(orig_preview)
    print("\nReshaped preview:")
    print(resh_preview)


SUCCESS: Reshaped Sheet 2 metadata for January 2018 matches the original.

Sample content verification (first 5 variables):

Variable: PUFREG_VS1
                                   Label min_value max_value additional_value
0                National Capital Region        13         0                0
1       Cordillera Administrative Region        14         0                0
2               Region I - Ilocos Region         1         0                0
3             Region II - Cagayan Valley         2         0                0
4             Region III - Central Luzon         3         0                0
5                Region IVA - CALABARZON         4         0                0
6                  Region IVB - MIMAROPA        17         0                0
7                        Region V- Bicol         5         0                0
8            Region VI - Western Visayas         6         0                0
9           Region VII - Central Visayas         7         0              

Checking for August 2024 Original vs Reshaped Metadata Sheet 2

In [22]:
# ===========================
# Caller: content-based verification with month/year in messages
# ===========================

# Load original metadata (Sheet 2)
original_df = load_dataset("2024", "August", "metadata", 1)

# Load reshaped metadata (automatically reshaped)
reshaped_df = load_dataset("2024", "August", "metadata", 1)

# Store month/year for messages
month = "August"
year = "2024"

# Call the verifier
result_message = verify_sheet2(original_df, reshaped_df)

# Customize message to include month/year
if "SUCCESS" in result_message:
    print(f"SUCCESS: Reshaped Sheet 2 metadata for {month} {year} matches the original.\n")
    print("Sample content verification (first 5 variables):\n")
    
    # Pick first 5 unique variables from reshaped
    sample_vars = reshaped_df["Variable"].unique()[:5]
    for var in sample_vars:
        sample_rows = reshaped_df[reshaped_df["Variable"] == var]
        print(f"Variable: {var}")
        print(sample_rows[["Label", "min_value", "max_value", "additional_value"]])
        print("-" * 50)
else:
    print(f"MISMATCH FOUND in {month} {year} metadata:\n")
    # Show first few rows from both original and reshaped for inspection
    orig_preview = original_df.fillna("").astype(str).head(10)
    resh_preview = reshaped_df.fillna("").astype(str).head(10)
    print("Original preview:")
    print(orig_preview)
    print("\nReshaped preview:")
    print(resh_preview)


SUCCESS: Reshaped Sheet 2 metadata for August 2024 matches the original.

Sample content verification (first 5 variables):

Variable: PUFSVYMO_VS1
        Label min_value max_value additional_value
0     January       1.0         0                0
1    February       2.0         0                0
2       March       3.0         0                0
3       April       4.0         0                0
4         May       5.0         0                0
5        June       6.0         0                0
6        July       7.0         0                0
7      August       8.0         0                0
8   September       9.0         0                0
9     October      10.0         0                0
10   November      11.0         0                0
11   December      12.0         0                0
--------------------------------------------------
Variable: PUFHHSIZE_VS1
   Label min_value max_value additional_value
12     1       1.0         0                0
13     2       2.0     

Checking January 2018 Metadata Reshaped Sheet 2

In [23]:
clean_jan_2018 = load_dataset("2018", "January", "metadata", 1)
clean_jan_2018.head()

,Variable,Description,Label,min_value,max_value,additional_value
0,PUFREG_VS1,Region,National Capital Region,13,0,0
1,PUFREG_VS1,Region,Cordillera Administrative Region,14,0,0
2,PUFREG_VS1,Region,Region I - Ilocos Region,1,0,0
3,PUFREG_VS1,Region,Region II - Cagayan Valley,2,0,0
4,PUFREG_VS1,Region,Region III - Central Luzon,3,0,0


Checking August 2024 Metadata Reshaped Sheet 2

In [24]:
clean_aug_2024 = load_dataset("2024", "August", "metadata", 1)
clean_aug_2024.head()

,Variable,Description,Label,min_value,max_value,additional_value
0,PUFSVYMO_VS1,Survey Month,January,1.0,0,0
1,PUFSVYMO_VS1,Survey Month,February,2.0,0,0
2,PUFSVYMO_VS1,Survey Month,March,3.0,0,0
3,PUFSVYMO_VS1,Survey Month,April,4.0,0,0
4,PUFSVYMO_VS1,Survey Month,May,5.0,0,0


Checking for Variables with Additional Value (Extra Column)

In [25]:
# Load metadata Sheet 2 for your desired month/year
df_metadata = load_dataset("2018", "January", "metadata", 1)

# Filter rows for PUFC10_CONWR_VS1
pu_fc10 = df_metadata[df_metadata["Variable"] == "PUFC10_CONWR_VS1"]

# Display the result
pu_fc10


,Variable,Description,Label,min_value,max_value,additional_value
219,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Overseas Contract Workers,1,0,0
220,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Workers other than OCW,2,0,0
221,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,"Employees in Philippine Embassy, Consulates & ...",3,0,0
222,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Students abroad/Tourists,4,0,0
223,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Others,5,0,0
224,PUFC10_CONWR_VS1,C10-Overseas Filipino Indicator,Less than 15 Years Old,0,0,Not Applicable


--------

## Decoding Survey Headers from Sheet 1

In [26]:
import os
import pandas as pd

# -------------------------------------------------------------------
# Decoded DataFrames for later use (Sheet 2 decoding)
# -------------------------------------------------------------------
decoded_dataframes = {}   


# ----------------------------------------------------------
# Load Metadata Sheet 1 for a given year/month
# ----------------------------------------------------------
def load_sheet1(year, month):
    folder = "Metadata Sheet 1 CSV's"
    filename = f"Sheet1_{month}_{year}.csv"
    path = os.path.join(base_path, folder, year, filename)

    if os.path.exists(path):
        return pd.read_csv(path)
    else:
        return None


# ----------------------------------------------------------
# Decode survey headers using Sheet 1 metadata
# ----------------------------------------------------------
def decode_survey_headers(survey_df, sheet1_df):
    mapping = dict(zip(sheet1_df['Variable'], sheet1_df['Description']))

    decoded = []
    unchanged = []

    new_columns = {}
    for col in survey_df.columns:
        if col in mapping:
            new_columns[col] = mapping[col]
            decoded.append(col)
        else:
            new_columns[col] = col
            unchanged.append(col)

    renamed_df = survey_df.rename(columns=new_columns)
    return renamed_df, decoded, unchanged


# ----------------------------------------------------------
# Run across ALL survey sheets and build ONE summary table
# ----------------------------------------------------------
def run_full_header_decoding():

    summary_rows = []   # Master summary table

    print("\n===============================")
    print("  AUTOMATED HEADER DECODER")
    print("===============================\n")

    all_pass = True  # Track if all sheets fully decoded

    for year in sorted(inventory.keys()):

        for month in sorted(
            inventory[year].keys(), key=lambda m: month_order.get(m, 99)
        ):

            # Identify survey file
            survey_files = [
                f for f in inventory[year][month]
                if f["filetype"] == "survey"
            ]
            if not survey_files:
                continue

            survey_filename = survey_files[0]["filename"]
            survey_path = os.path.join(base_path, year, survey_filename)

            # Load raw survey CSV with low_memory=False to avoid DtypeWarning
            try:
                survey_df = pd.read_csv(survey_path, low_memory=False)
            except Exception as e:
                print(f"{year} {month}: ERROR loading CSV → {e}")
                continue

            # Load metadata sheet1
            sheet1_df = load_sheet1(year, month)

            if sheet1_df is None:
                summary_rows.append({
                    "Year": year,
                    "Month": month,
                    "Raw Headers Count": len(survey_df.columns),
                    "Decoded Headers Count": 0,
                    "Unchanged": len(survey_df.columns),
                    "Integrity": "NO METADATA"
                })
                all_pass = False
                continue

            # Decode headers
            decoded_df, decoded_cols, unchanged_cols = decode_survey_headers(
                survey_df, sheet1_df
            )

            # -----------------------------------------
            # NEW: Save decoded dataframe for later use
            # -----------------------------------------
            decoded_dataframes[(year, month)] = decoded_df

            # Integrity check
            integrity = "PASS" if len(unchanged_cols) == 0 else "FAIL"
            if integrity == "FAIL":
                all_pass = False

            # Store in summary
            summary_rows.append({
                "Year": year,
                "Month": month,
                "Raw Headers Count": len(survey_df.columns),
                "Decoded Headers Count": len(decoded_cols),
                "Unchanged": len(unchanged_cols),
                "Integrity": integrity
            })

    # Convert summary to DataFrame
    summary_df = pd.DataFrame(summary_rows)

    if all_pass:
        print("SUCCESS: All survey sheet headers are decoded.\n")
    else:
        print("Some survey sheet headers were not fully decoded. Check the summary.\n")

    print("\n===============================")
    print("     FINAL DECODING SUMMARY")
    print("===============================\n")
    display(summary_df)

    return summary_df


# ----------------------------------------------------------
# RUN
# ----------------------------------------------------------
summary = run_full_header_decoding()



  AUTOMATED HEADER DECODER

SUCCESS: All survey sheet headers are decoded.


     FINAL DECODING SUMMARY



,Year,Month,Raw Headers Count,Decoded Headers Count,Unchanged,Integrity
0,2018,January,50,50,0,PASS
1,2018,April,50,50,0,PASS
2,2018,July,51,51,0,PASS
3,2018,October,51,51,0,PASS
4,2019,January,49,49,0,PASS
5,2019,April,49,49,0,PASS
6,2019,July,49,49,0,PASS
7,2019,October,49,49,0,PASS
8,2022,January,52,52,0,PASS
9,2022,February,41,41,0,PASS


In [27]:
decoded_dataframes[("2018", "January")]

,Region,Province,Province Recode,Household Unique Sequential Number,2010Urban-RuralFIES,Final Weight Based on Projection (provincial projections),Survey Month,Survey Year,Psu Number,Replicate,...,C33-Number of Weeks Spent in Looking for Work,C34-Reason for not Looking for Work,C35-When Last Looked for Work,C36-Available for Work,C37-Willingness to take up work during the past week or withing two weeks,C38-Previous Job Indicator,C40-Previous Occupation,C41-Did work or had a job during the past quarter,C43-Kind of Business (past quarter),"New Employment Criteria (jul 05, 2005)"
0,14,1,100,1,2,124.9425,1,2018,140,32,...,,6,,,,1,52,2,,3
1,14,1,100,1,2,131.2126,1,2018,140,32,...,,,,,,,,1,01,1
2,14,1,100,1,2,142.0464,1,2018,140,32,...,,,,,,,,1,01,1
3,14,1,100,1,2,138.2958,1,2018,140,32,...,,,,,,,,,,
4,14,1,100,2,2,195.4152,1,2018,140,32,...,,,,,,,,1,41,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180257,12,98,9804,41313,2,378.1809,1,2018,111,8,...,,,,,,,,,,
180258,12,98,9804,41314,2,371.0326,1,2018,111,8,...,,,,,,,,1,49,1
180259,12,98,9804,41314,2,415.4080,1,2018,111,8,...,,,,,,,,,,
180260,12,98,9804,41314,2,378.1809,1,2018,111,8,...,,,,,,,,,,


In [28]:
decoded_dataframes[("2024", "August")]

,Household Unique Sequential Number,Final Weight Based on Projection,Survey Month,Survey Year,Psu Number,Replicate,Household Size,C101-Line Number,C03-Relationship to Household Head,C04-Sex,...,C25-Looked for Work or Tried to Establish Business during the past week,C25B - First time to look for work,C26-Reason for not Looking for Work,C27-Available for Work,C28-Previous Job Indicator,C29 - Last worked (Year),C29 - Last worked (Month),C31-Previous Occupation,C33-Kind of Business (previous occupation),"New Employment Criteria (jul 05, 2005)"
0,1,1368.0227,8,2024,517,61,3,1,1,1,...,,,,,,,,,,1
1,1,1804.6674,8,2024,517,61,3,2,3,1,...,,,,,,,,,,
2,1,1251.5455,8,2024,517,61,3,3,2,2,...,,,,,,,,,,
3,2,1088.9678,8,2024,517,61,2,1,1,1,...,,,,,,,,,,1
4,2,1368.0227,8,2024,517,61,2,2,3,1,...,,,,,,,,,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43370,11093,921.8419,8,2024,147,37,9,7,3,1,...,,,,,,,,,,
43371,11093,921.8419,8,2024,147,37,9,8,3,1,...,,,,,,,,,,
43372,11093,921.8419,8,2024,147,37,9,9,3,1,...,,,,,,,,,,
43373,11094,665.4090,8,2024,147,37,2,1,1,1,...,,,,,,,,,,1


### Coverage Scanner in Metadata and Survey 

In [29]:
def scan_sheet2_coverage_all_print(decoded_dataframes, load_sheet2_fn):
    """
    Loops over all decoded survey DataFrames and prints a Sheet 2 coverage report.
    
    Args:
        decoded_dataframes : dict
            {(year, month): decoded survey DataFrame}
        load_sheet2_fn : function
            Function to load Sheet 2 metadata for a given year/month.
    """
    for (year, month), decoded_survey_df in decoded_dataframes.items():
        # Load corresponding Sheet 2 metadata
        try:
            sheet2_df = load_sheet2_fn(year, month)
        except FileNotFoundError:
            print(f"\n⚠️ Sheet 2 metadata not found for {month} {year}. Skipping...")
            continue

        # Survey columns (decoded headers → Descriptions)
        survey_vars = set(decoded_survey_df.columns)
        # Metadata Descriptions
        metadata_descriptions = set(sheet2_df['Description'].unique())

        # Compare
        unmatched_survey_vars = sorted(list(survey_vars - metadata_descriptions))
        unmatched_metadata_vars = sorted(list(metadata_descriptions - survey_vars))

        # Print report
        print(f"\n--- Sheet 2 Coverage Scanner for {month} {year} ---")
        
        if unmatched_survey_vars:
            print("⚠️ Columns in survey but NOT in metadata Description:")
            for col in unmatched_survey_vars:
                print("  -", col)
        else:
            print("✅ All survey columns exist in Sheet 2 metadata Descriptions.")
        
        if unmatched_metadata_vars:
            print("⚠️ Descriptions in metadata but NOT in survey:")
            for desc in unmatched_metadata_vars:
                print("  -", desc)
        else:
            print("✅ All metadata Descriptions exist in the survey.")

In [30]:
scan_sheet2_coverage_all_print(
    decoded_dataframes, 
    load_sheet2_fn=lambda y, m: load_dataset(y, m, filetype="metadata", sheet_number=1)
)


--- Sheet 2 Coverage Scanner for January 2018 ---
⚠️ Columns in survey but NOT in metadata Description:
  - Final Weight Based on Projection (provincial projections)
  - Household Unique Sequential Number
  - Province
  - Province Recode
  - Psu Number
  - Replicate
  - Survey Year
⚠️ Descriptions in metadata but NOT in survey:
  - C07-Age (15 & Over)
  - C07-Age as of Last Birthday

--- Sheet 2 Coverage Scanner for April 2018 ---
⚠️ Columns in survey but NOT in metadata Description:
  - C05-Age as of Last Birthday
  - Final Weight Based on Projection (provincial projections)
  - Psu Number
  - Replicate
  - Survey Year
  - household_seq_number
  - province
  - province_recode
⚠️ Descriptions in metadata but NOT in survey:
  - C07-Age as of Last Birthday

--- Sheet 2 Coverage Scanner for July 2018 ---
⚠️ Columns in survey but NOT in metadata Description:
  - C05-Age as of Last Birthday
  - C07-Highest Grade Completed
  - Final Weight Based on Projection (provincial projections)
  - Ho

### Values Decoding (Samples)

For Single-Value Codes

In [31]:
YEAR = "2018"
MONTH = "January"

# Get the survey DataFrame with decoded headers (Descriptions)
decoded_survey = decoded_dataframes[(YEAR, MONTH)]

# Focused check: show a few key decoded columns before value decoding
columns_to_check = ["Region", "Survey Month"]
existing_columns = [c for c in columns_to_check if c in decoded_survey.columns]

# Display the first 10 rows
display(decoded_survey[existing_columns].head())

,Region,Survey Month
0,14,1
1,14,1
2,14,1
3,14,1
4,14,1


In [32]:
import os
import pandas as pd

# ---------------------------------
# Function: Load Clean Sheet 2
# ---------------------------------
def load_clean_sheet2(year, month):
    """
    Loads the pre-processed clean Sheet 2 metadata (values dictionary) CSV.
    """
    main_folder_name = "Metadata Sheet 2 CSV's"
    filename = f"Sheet2_{month}_{year}.csv"
    file_path = os.path.join(base_path, main_folder_name, year, filename)
    
    # Fallback
    if not os.path.exists(file_path):
        file_path_fallback = os.path.join(base_path, year, filename)
        if os.path.exists(file_path_fallback):
            file_path = file_path_fallback
        else:
            raise FileNotFoundError(f"Not found: {file_path}")

    return pd.read_csv(
        file_path,
        dtype={
            'min_value': str,
            'max_value': str,
            'additional_value': str
        }
    )

# ---------------------------------
# FAST single-value decoder (no _LABEL)
# ---------------------------------
def decode_single_variable_from_description(survey_df, clean_metadata_df, description_name):
    """
    Decodes a single survey column by directly replacing the values in the column.
    
    """

    # Filter metadata
    lookup = clean_metadata_df[
        clean_metadata_df["Description"].str.strip().str.lower() ==
        description_name.strip().lower()
    ].copy()

    if lookup.empty:
        print(f"⚠️ No metadata for: {description_name}")
        return survey_df

    # Build mapping dictionary (string-based)
    lookup["min_value"] = lookup["min_value"].astype(str).str.strip()
    mapping_dict = dict(zip(lookup["min_value"], lookup["Label"]))

    # Apply fast replace
    survey_df[description_name] = (
        survey_df[description_name]
        .astype(str)
        .str.strip()
        .replace(mapping_dict)
    )

    return survey_df


In [33]:
YEAR = "2018"
MONTH = "January"

decoded_survey = decoded_dataframes[(YEAR, MONTH)]
sheet2_clean = load_clean_sheet2(YEAR, MONTH)

decoded_survey = decode_single_variable_from_description(decoded_survey, sheet2_clean, "Region")
decoded_survey = decode_single_variable_from_description(decoded_survey, sheet2_clean, "Survey Month")

decoded_survey[["Region", "Survey Month"]].head()

,Region,Survey Month
0,Cordillera Administrative Region,January
1,Cordillera Administrative Region,January
2,Cordillera Administrative Region,January
3,Cordillera Administrative Region,January
4,Cordillera Administrative Region,January


In [34]:
def check_value_decoding_integrity(df, variables):
    """
    Integrity Checker:
    - Raw Total Records: Count of non-null rows
    - Decoded Total Records: Count of non-numeric rows (meaning decoded)
    - PASS if decoded_total == raw_total (fully decoded)
    """

    records = []

    for var in variables:

        if var not in df.columns:
            records.append([var, 0, 0, "MISSING COLUMN"])
            continue

        # Raw total = all non-null rows
        raw_total = df[var].notna().sum()

        # Decoded total = non-numeric rows
        decoded_total = df[var].apply(lambda x: not str(x).strip().isdigit()).sum()

        # PASS if everything is decoded
        status = "PASS" if decoded_total == raw_total else "FAIL"

        records.append([var, raw_total, decoded_total, status])

    return pd.DataFrame(
        records,
        columns=["Variable", "Raw Total Records", "Decoded Total Records", "Integrity Status"]
    )

In [35]:
YEAR = "2018"
MONTH = "January"

df = decoded_dataframes[(YEAR, MONTH)]  # has decoded headers; apply Sheet 2 decoding first

variables_to_check = ["Region", "Survey Month"]

integrity_report = check_value_decoding_integrity(df, variables_to_check)
integrity_report

,Variable,Raw Total Records,Decoded Total Records,Integrity Status
0,Region,180262,180262,PASS
1,Survey Month,180262,180262,PASS


For Ranged Values

In [36]:
YEAR = "2018"
MONTH = "January"

# Load the original survey dataframe (decoded headers but NOT value-decoded)
original_survey = decoded_dataframes[(YEAR, MONTH)]

# Print only the original C14 column (first 10 rows)
display(original_survey[["C14-Primary Occupation"]].head(10))

,C14-Primary Occupation
0,
1,61
2,92
3,
4,93
5,52
6,
7,
8,
9,72


In [37]:
def decode_ranged_variable_from_description(survey_df, clean_metadata_df, description_name):
    """
    Decodes a survey column (already using the Description as header)
    using the clean Sheet 2 metadata, handling both single values and ranged values.

    """

    # Filter metadata rows for this description
    lookup_table = clean_metadata_df[
        clean_metadata_df['Description'].str.strip().str.lower() == description_name.strip().lower()
    ].copy()

    if lookup_table.empty:
        print(f"⚠️ Warning: No metadata found for '{description_name}'")
        return survey_df

    # Convert min/max to numeric
    lookup_table['min_value'] = pd.to_numeric(lookup_table['min_value'], errors='coerce')
    lookup_table['max_value'] = pd.to_numeric(lookup_table['max_value'], errors='coerce')

    def map_value(val):
        try:
            val_num = float(val)
        except (ValueError, TypeError):
            return val  # keep original if not numeric

        # Exact match for single values
        exact = lookup_table[
            (lookup_table['min_value'] == val_num) &
            (lookup_table['max_value'].isna() | (lookup_table['max_value'] == 0))
        ]
        if not exact.empty:
            return exact['Label'].values[0]

        # Range match
        for _, row in lookup_table.iterrows():
            if pd.notna(row['min_value']) and pd.notna(row['max_value']):
                if row['min_value'] <= val_num <= row['max_value']:
                    return row['Label']

        return val

    # Overwrite the original column
    survey_df[description_name] = survey_df[description_name].apply(map_value)

    return survey_df


In [38]:
DESCRIPTION = "C14-Primary Occupation"

decoded_survey = decode_ranged_variable_from_description(
    decoded_survey, sheet2_clean, DESCRIPTION
)

# Print only the decoded column
decoded_survey[[DESCRIPTION]].head(10)

,C14-Primary Occupation
0,
1,"Skilled Agricultural, Forestry and Fishery Wor..."
2,Elementary Occupations
3,
4,Elementary Occupations
5,Service and Sales Workers
6,
7,
8,
9,Craft and Related Trades Workers


In [39]:
def check_decoded_integrity(df, variables):
    records = []

    for var in variables:
        if var not in df.columns:
            records.append([var, 0, 0, "MISSING COLUMN"])
            continue

        raw_total = df[var].notna().sum()

        # Count decoded rows = rows that are NOT numeric anymore
        decoded_total = df[var].apply(lambda x: not str(x).isdigit()).sum()

        status = "PASS" if decoded_total > 0 else "FAIL"

        records.append([var, raw_total, decoded_total, status])

    return pd.DataFrame(
        records,
        columns=["Variable", "Raw Total Records", "Decoded Total Records", "Integrity Status"]
    )


In [40]:
# Integrity check
variables_to_check = [DESCRIPTION]
integrity_report_ranged = check_decoded_integrity(decoded_survey, variables_to_check)

integrity_report_ranged

,Variable,Raw Total Records,Decoded Total Records,Integrity Status
0,C14-Primary Occupation,180262,180262,PASS


Checking if "Not Applicable" value exists in all survey sheets

In [41]:
def scan_not_applicable_all(decoded_dataframes):
    """
    Loops over all decoded survey DataFrames and checks if "Not Applicable" exists.
    
    Returns:
        report_df : DataFrame with columns: Year, Month, 'Not Applicable' Count, Status.
    Prints a message on top if no 'Not Applicable' values exist in all sheets.
    """
    report_rows = []

    for (year, month), df in decoded_dataframes.items():
        # Count all occurrences of "Not Applicable" in the entire DataFrame
        na_count = (df == "Not Applicable").sum().sum()
        status = "FOUND" if na_count > 0 else "NONE"

        # Store results
        report_rows.append({
            "Year": year,
            "Month": month,
            "'Not Applicable' Count": na_count,
            "Status": status
        })

    # Convert to DataFrame for a summary table
    report_df = pd.DataFrame(report_rows)

    # Check if all counts are 0
    if report_df["'Not Applicable' Count"].sum() == 0:
        print("All survey sheets have no 'Not Applicable' value\n")

    return report_df

In [42]:
not_applicable_report = scan_not_applicable_all(decoded_dataframes)
display(not_applicable_report)

All survey sheets have no 'Not Applicable' value



,Year,Month,'Not Applicable' Count,Status
0,2018,January,0,NONE
1,2018,April,0,NONE
2,2018,July,0,NONE
3,2018,October,0,NONE
4,2019,January,0,NONE
5,2019,April,0,NONE
6,2019,July,0,NONE
7,2019,October,0,NONE
8,2022,January,0,NONE
9,2022,February,0,NONE


### Automated Value Decoder

Test Case: January 2018

In [43]:
import pandas as pd
import numpy as np

# -----------------------------
# VECTOR-BASED DECODER FOR ONE COLUMN
# -----------------------------
def decode_column_vectorized(df, metadata, description_name):
    """
    Decodes a single column in the survey DataFrame using metadata.
    Handles:
      1. Additional values
      2. Single-value codes (min_value only, max_value NaN or 0)
      3. Numeric ranges (min_value to max_value)
    """
    if description_name not in df.columns:
        return df

    # Filter metadata for this column
    lookup = metadata[
        metadata["Description"].str.strip().str.lower() == description_name.strip().lower()
    ].copy()
    if lookup.empty:
        return df

    col = df[description_name].astype(str).str.strip()

    # 1️⃣ Additional values mapping
    add_map = lookup[lookup["additional_value"].notna()]
    if not add_map.empty:
        add_dict = dict(zip(add_map["additional_value"].astype(str).str.strip(), add_map["Label"]))
        mask_add = col.isin(add_dict)
        col.loc[mask_add] = col.loc[mask_add].map(add_dict)

    # 2️⃣ Single-value codes (min_value only, max_value NaN or 0)
    single_val = lookup[(lookup["max_value"].isna()) | (lookup["max_value"] == "0")]
    for min_val, label in zip(single_val["min_value"], single_val["Label"]):
        mask = col == str(min_val).strip()
        col.loc[mask] = label

    # 3️⃣ Range matches (numeric)
    ranges = lookup[lookup["max_value"].notna() & lookup["min_value"].notna() & (lookup["max_value"] != "0")]
    if not ranges.empty:
        col_numeric = pd.to_numeric(col, errors="coerce")
        for min_val, max_val, label in zip(ranges["min_value"].astype(float),
                                           ranges["max_value"].astype(float),
                                           ranges["Label"]):
            mask = (col_numeric >= min_val) & (col_numeric <= max_val)
            col.loc[mask] = label

    df[description_name] = col.astype(object)
    return df


# -----------------------------
# DECODE ALL COLUMNS (VECTORIZED)
# -----------------------------
def decode_all_values_vectorized(survey_df, metadata_df):
    """Apply vectorized decoding to all columns in the survey DataFrame."""
    for col in survey_df.columns:
        survey_df = decode_column_vectorized(survey_df, metadata_df, col)
    return survey_df


In [44]:
# -----------------------------
# USAGE EXAMPLE: January 2018
# -----------------------------
YEAR = "2018"
MONTH = "January"

# Fresh copy of the survey DataFrame (decoded headers only)
fresh_survey_df = decoded_dataframes[(YEAR, MONTH)].copy()

# Load clean Sheet 2 metadata
sheet2_clean = load_clean_sheet2(YEAR, MONTH)

# Decode all columns
decoded_survey_full = decode_all_values_vectorized(fresh_survey_df, sheet2_clean)

# Preview 
decoded_survey_full


,Region,Province,Province Recode,Household Unique Sequential Number,2010Urban-RuralFIES,Final Weight Based on Projection (provincial projections),Survey Month,Survey Year,Psu Number,Replicate,...,C33-Number of Weeks Spent in Looking for Work,C34-Reason for not Looking for Work,C35-When Last Looked for Work,C36-Available for Work,C37-Willingness to take up work during the past week or withing two weeks,C38-Previous Job Indicator,C40-Previous Occupation,C41-Did work or had a job during the past quarter,C43-Kind of Business (past quarter),"New Employment Criteria (jul 05, 2005)"
0,Cordillera Administrative Region,1,100,1,Rural,124.9425,January,2018,140,32,...,,Too young/old or Retired/Permanent Disability,,,,yes,Service and Sales Workers,no,,NOT IN THE LABOR FORCE
1,Cordillera Administrative Region,1,100,1,Rural,131.2126,January,2018,140,32,...,,,,,,,,yes,Agriculture and Forestry,EMPLOYED
2,Cordillera Administrative Region,1,100,1,Rural,142.0464,January,2018,140,32,...,,,,,,,,yes,Agriculture and Forestry,EMPLOYED
3,Cordillera Administrative Region,1,100,1,Rural,138.2958,January,2018,140,32,...,,,,,,,,,,
4,Cordillera Administrative Region,1,100,2,Rural,195.4152,January,2018,140,32,...,,,,,,,,yes,Construction,EMPLOYED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180257,Region XII - SOCCSKSARGEN,98,9804,41313,Rural,378.1809,January,2018,111,8,...,,,,,,,,,,
180258,Region XII - SOCCSKSARGEN,98,9804,41314,Rural,371.0326,January,2018,111,8,...,,,,,,,,yes,Transportation and Storage,EMPLOYED
180259,Region XII - SOCCSKSARGEN,98,9804,41314,Rural,415.4080,January,2018,111,8,...,,,,,,,,,,
180260,Region XII - SOCCSKSARGEN,98,9804,41314,Rural,378.1809,January,2018,111,8,...,,,,,,,,,,


In [45]:
# -----------------------------
# Save the fully decoded survey to CSV
# -----------------------------
output_folder = os.path.join(base_path, "Decoded Surveys", YEAR)
os.makedirs(output_folder, exist_ok=True)

output_filename = f"Survey_Decoded_{MONTH}_{YEAR}.csv"
output_path = os.path.join(output_folder, output_filename)

# Save as CSV
decoded_survey_full.to_csv(output_path, index=False)

print(f"✅ Fully decoded survey saved to: {output_path}")

✅ Fully decoded survey saved to: G:\.shortcut-targets-by-id\1VctTphaltRx4xcPxmTJlRTrxLalyuEt8\Labor Force Survey\Decoded Surveys\2018\Survey_Decoded_January_2018.csv


#### Verifier to check whether values were encoded or not

In [46]:
def verify_decoding(before_df, after_df, metadata_df):
    records = []

    # Columns that SHOULD be decoded (present in metadata)
    metadata_cols = (
        metadata_df["Description"]
        .astype(str)
        .str.strip()
        .str.lower()
        .unique()
    )

    for col in before_df.columns:

        # 1. Check if this column has metadata
        in_metadata = col.strip().lower() in metadata_cols

        if not in_metadata:
            records.append({
                "Variable": col,
                "In_Metadata?": False,
                "Decoded?": False,
                "Status": "No metadata"
            })
            continue

        # 2. Compare BEFORE and AFTER values
        before_vals = before_df[col].astype(str).fillna("NA")
        after_vals = after_df[col].astype(str).fillna("NA")

        changed = not before_vals.equals(after_vals)

        if changed:
            status = "Decoded"
        else:
            status = "Retained (no matching metadata value)"

        records.append({
            "Variable": col,
            "In_Metadata?": True,
            "Decoded?": changed,
            "Status": status
        })

    return pd.DataFrame(records)


In [47]:
YEAR = "2018"
MONTH = "January"

# BEFORE decoding (headers decoded only, values still raw)
before_df = decoded_dataframes[(YEAR, MONTH)].copy()

# Load metadata
sheet2_clean = load_clean_sheet2(YEAR, MONTH)

# AFTER decoding (values processed)
after_df = decode_all_values_vectorized(before_df.copy(), sheet2_clean)

# Run verifier
report = verify_decoding(before_df, after_df, sheet2_clean)

report


,Variable,In_Metadata?,Decoded?,Status
0,Region,True,False,Retained (no matching metadata value)
1,Province,False,False,No metadata
2,Province Recode,False,False,No metadata
3,Household Unique Sequential Number,False,False,No metadata
4,2010Urban-RuralFIES,True,True,Decoded
5,Final Weight Based on Projection (provincial p...,False,False,No metadata
6,Survey Month,True,False,Retained (no matching metadata value)
7,Survey Year,False,False,No metadata
8,Psu Number,False,False,No metadata
9,Replicate,False,False,No metadata
